In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
import json

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
33,application_1614556309029_0034,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
%%configure -f
{"executorMemory": "10000M", "executorCores": 4}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
35,application_1614556309029_0036,pyspark,idle,Link,Link,


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import *

ss = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
sc = ss.sparkContext.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Analytics Goals

Will be doing classification analysis of tracks from the Million Songs DataSet by predicting the genre and sentiment of a track_id based on information such as the top tags and how similar it is with other songs. The results of this classification will be compared with the predictions using lyrics to see if there is an improvement or not with using these features. 

I only used track_ids that have atleast three tags and no none values with similarities. So I just ignored other missing data

## Last.Fm Dataset

Need this dataset to get information about tags and similarity for each song to other songs. Will be used to predict the song's sentiment and genre and see if it makes accurate predictions depending on that info. 

In [18]:
def preprocess_fm_for_genre(files_rdd): 
    files_contents = files_rdd.map(lambda x: x[1]).map(lambda x: json.loads(x))\
                .map(lambda x: (x['track_id'], x['title'], x['artist'], 
                                sorted(x['tags'], key=lambda y: y[1]), 
                                sorted(x['similars'], key=lambda y: y[1])))\
                .filter(lambda x: len(x[3]) >= 3).filter(lambda x: len(x[4]) >= 10)\
                .map(lambda x: (x[0], x[1], x[2], x[3][0][0], x[3][1][0], x[3][2][0], x[4][0][0], 
                               x[4][1][0], x[4][2][0], x[4][3][0], x[4][4][0], x[4][5][0], 
                               x[4][6][0], x[4][7][0], x[4][8][0], x[4][9][0]))
    schema = StructType([
    StructField('Track_ID', StringType(), True), 
    StructField('Title', StringType(), True), 
    StructField('Artist', StringType(), True), 
    StructField('Tag_1', StringType(), True),
    StructField('Tag_2', StringType(), True), 
    StructField('Tag_3', StringType(), True), 
    StructField('Similar_1', StringType(), True), 
    StructField('Similar_2', StringType(), True), 
    StructField('Similar_3', StringType(), True), 
    StructField('Similar_4', StringType(), True), 
    StructField('Similar_5', StringType(), True), 
    StructField('Similar_6', StringType(), True), 
    StructField('Similar_7', StringType(), True), 
    StructField('Similar_8', StringType(), True), 
    StructField('Similar_9', StringType(), True), 
    StructField('Similar_10', StringType(), True)
    ])
    file_df = ss.createDataFrame(files_contents, schema)
    return file_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
import json
file_list = ["Z"]
for f in file_list: 
    input_train = "s3://msds694-final-group7/lastfm_train/" + f + "/*/*/"
    input_test = "s3://msds694-final-group7/lastfm_test/" + f + "/*/*/"
    
    files_rdd_train = sc.wholeTextFiles(input_train)
    files_rdd_test = sc.wholeTextFiles(input_test)
    
    train_df = preprocess_fm_for_genre(files_rdd_train)
    test_df = preprocess_fm_for_genre(files_rdd_test)
    
    output_train_file = "s3a://msds694-final-group7/lastfm_train_for_genre_clean/" + f
    output_test_file = "s3a://msds694-final-group7/lastfm_test_for_genre_clean/" + f
    train_df.write.parquet(output_train_file)
    test_df.write.parquet(output_test_file)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
train_parquets_path = "s3a://msds694-final-group7/lastfm_train_for_genre_clean/*"
test_parquets_path = "s3a://msds694-final-group7/lastfm_test_for_genre_clean/*"
train = ss.read.parquet(train_parquets_path)
test = ss.read.parquet(test_parquets_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
train.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----+------+-----+-------+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|          Track_ID|Title|Artist|Tag_1|  Tag_2|  Tag_3|         Similar_1|         Similar_2|         Similar_3|         Similar_4|         Similar_5|         Similar_6|         Similar_7|         Similar_8|         Similar_9|        Similar_10|
+------------------+-----+------+-----+-------+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|TRCAAAO128E078E7A9|Games| Big L|  rap|Hip-Hop|classic|TRNYCZP128E07924EE|TRYDRBS128F92FCBB4|TRSOCXU128E0796974|TRDRHVG128F92FAB06|TRWJHHQ128F4215F03|TRJQZWT128F930BD04|TRTFJKQ128F9321D43|TRAYSZT128F423B584|TRXLTEB128F4279D0C|TRMILWY128F429D9F8|
+---------------

In [21]:
train.write.parquet("s3a://msds694-final-group7/lastfm_train_for_genre_clean/All")
test.write.parquet("s3a://msds694-final-group7/lastfm_test_for_genre_clean/All")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

u'path s3a://msds694-final-group7/lastfm_test_for_genre_clean/All already exists.;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 843, in parquet
    self._jwrite.parquet(path)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
AnalysisException: u'path s3a://msds694-final-group7/lastfm_test_for_genre_clean/All already exists.;'



In [9]:
train.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----+------+-----+-------+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|          Track_ID|Title|Artist|Tag_1|  Tag_2|  Tag_3|         Similar_1|         Similar_2|         Similar_3|         Similar_4|         Similar_5|         Similar_6|         Similar_7|         Similar_8|         Similar_9|        Similar_10|
+------------------+-----+------+-----+-------+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|TRCAAAO128E078E7A9|Games| Big L|  rap|Hip-Hop|classic|TRNYCZP128E07924EE|TRYDRBS128F92FCBB4|TRSOCXU128E0796974|TRDRHVG128F92FAB06|TRWJHHQ128F4215F03|TRJQZWT128F930BD04|TRTFJKQ128F9321D43|TRAYSZT128F423B584|TRXLTEB128F4279D0C|TRMILWY128F429D9F8|
+---------------

## Genre Labels

Get the genre labels for each track id

In [23]:
genre_rdd = sc.textFile("s3://msds694-final-group7/msd-MAGD-genreAssignment.cls")
genre_rdd = genre_rdd.map(lambda x: x.split('\t')).map(lambda x: (str(x[0]), str(x[1])))
schema = StructType([
    StructField('track_id', StringType(), True), 
    StructField('genre', StringType(), True)
])
genre_df = ss.createDataFrame(genre_rdd, schema)
genre_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------+
|          track_id|         genre|
+------------------+--------------+
|TRAAAAK128F9318786|      Pop_Rock|
|TRAAAAV128F421A322|      Pop_Rock|
|TRAAAAW128F429D538|           Rap|
|TRAAABD128F429CF47|      Pop_Rock|
|TRAAACV128F423E09E|      Pop_Rock|
|TRAAADT12903CCC339|Easy_Listening|
|TRAAAED128E0783FAB|         Vocal|
|TRAAAEF128F4273421|      Pop_Rock|
|TRAAAEM128F93347B9|    Electronic|
|TRAAAFD128F92F423A|      Pop_Rock|
|TRAAAFP128F931B4E3|           Rap|
|TRAAAGR128F425B14B|      Pop_Rock|
|TRAAAGW12903CC1049|         Blues|
|TRAAAHD128F42635A5|      Pop_Rock|
|TRAAAHE12903C9669C|      Pop_Rock|
|TRAAAHJ128F931194C|      Pop_Rock|
|TRAAAHZ128E0799171|           Rap|
|TRAAAIR128F1480971|           RnB|
|TRAAAJG128F9308A25|          Folk|
|TRAAAMO128F1481E7F|     Religious|
+------------------+--------------+
only showing top 20 rows

## Combine last.fm and genre

Now put genre information in with the last_fm dataset as a label that will be used in the ml portion

In [25]:
last_fm_predict_genre = train.join(genre_df, on=['Track_ID'])
last_fm_predict_genre.write.parquet("s3a://msds694-final-group7/lastfm_train_for_genre_clean/All_with_genre")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
last_fm_predict_genre = test.join(genre_df, on=['Track_ID'])
last_fm_predict_genre.write.parquet("s3a://msds694-final-group7/lastfm_test_for_genre_clean/All_with_genre")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Sentiments

Get the sentiments for each track_id

In [35]:
mood_song = ss.read.csv('s3://msds694-final-group7/train_lyrics_rem_1000.txt', header=True)
mood_song.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+---------------------------------+-------------------------------+------+-----+----+
|file                 |artist                           |title                          |lyrics|mood |year|
+---------------------+---------------------------------+-------------------------------+------+-----+----+
|TRAAAAW128F429D538.h5|Casual                           |I Didn't Mean To               |null  |sad  |1994|
|TRAAAEF128F4273421.h5|Adam Ant                         |Something Girls                |null  |happy|1982|
|TRAAAFD128F92F423A.h5|Gob                              |Face the Ashes                 |null  |sad  |2007|
|TRAABJV128F1460C49.h5|Lionel Richie                    |Tonight Will Be Alright        |null  |happy|1986|
|TRAABLR128F423B7E3.h5|Blue Rodeo                       |Floating                       |null  |sad  |1987|
|TRAABVM128F92CA9DC.h5|Tesla                            |Caught In A Dream              |null  |happy|2004|
|TRAACER128F4290F96.h5|The D

Take out the ".h5" at the end of the file

In [49]:
def get_id(x): 
    return x.split('.')[0]
id_g = udf(get_id)
mood_song = mood_song.withColumn("Track_ID_file", id_g("file")).withColumnRenamed("title", "title_song")
mood_song.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+------+-----+----+------------------+
|                file|              artist|          title_song|lyrics| mood|year|     Track_ID_file|
+--------------------+--------------------+--------------------+------+-----+----+------------------+
|TRAAAAW128F429D53...|              Casual|    I Didn't Mean To|  null|  sad|1994|TRAAAAW128F429D538|
|TRAAAEF128F427342...|            Adam Ant|     Something Girls|  null|happy|1982|TRAAAEF128F4273421|
|TRAAAFD128F92F423...|                 Gob|      Face the Ashes|  null|  sad|2007|TRAAAFD128F92F423A|
|TRAABJV128F1460C4...|       Lionel Richie|Tonight Will Be A...|  null|happy|1986|TRAABJV128F1460C49|
|TRAABLR128F423B7E...|          Blue Rodeo|            Floating|  null|  sad|1987|TRAABLR128F423B7E3|
|TRAABVM128F92CA9D...|               Tesla|   Caught In A Dream|  null|happy|2004|TRAABVM128F92CA9DC|
|TRAACER128F4290F9...|The Dillinger Esc...|Setting Fire to S...|  null|happy|2004|

## Combine last.fm with Sentiments

Have sentiments as the label to the last_fm information

In [51]:
last_fm_predict_mood = train.join(mood_song, train['Track_ID'] == mood_song['Track_ID_file'])
last_fm_predict_mood = last_fm_predict_mood.drop("Track_ID_file", "artist", "lyrics", "file", "title_song")
last_fm_predict_mood.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----+----+
|          Track_ID|               Title|               Tag_1|               Tag_2|               Tag_3|         Similar_1|         Similar_2|         Similar_3|         Similar_4|         Similar_5|         Similar_6|         Similar_7|         Similar_8|         Similar_9|        Similar_10| mood|year|
+------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----+----+
|TRBAAGQ128F148D17E|         Crackbabies|          baby songs|                  ls

In [52]:
last_fm_predict_mood.write.parquet("s3a://msds694-final-group7/lastfm_train_for_sentiment_clean_All")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
last_fm_predict_mood_test = test.join(mood_song, test['Track_ID'] == mood_song['Track_ID_file'])
last_fm_predict_mood_test = last_fm_predict_mood_test.drop("Track_ID_file", "artist", "lyrics", "file", "title_song")
last_fm_predict_mood_test.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----+----+
|Track_ID|Title|Tag_1|Tag_2|Tag_3|Similar_1|Similar_2|Similar_3|Similar_4|Similar_5|Similar_6|Similar_7|Similar_8|Similar_9|Similar_10|mood|year|
+--------+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----+----+
+--------+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----+----+

In [58]:
mood_song.select(['Track_ID_file']).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|     Track_ID_file|
+------------------+
|TRAAAAW128F429D538|
|TRAAAEF128F4273421|
|TRAAAFD128F92F423A|
|TRAABJV128F1460C49|
|TRAABLR128F423B7E3|
|TRAABVM128F92CA9DC|
|TRAACER128F4290F96|
|TRAACFV128F935E50B|
|TRAACHN128F1489601|
|TRAACNS128F14A2DF5|
|TRAACQT128F9331780|
|TRAACTB12903CAAF15|
|TRAADLN128F14832E9|
|TRAADNA128F9331246|
|TRAADQX128F422B4CF|
|TRAADYB128F92D7E73|
|TRAADYI128E078FB38|
|TRAAEAM128F934B199|
|TRAAEEH128E0795DFE|
|TRAAEUB12903CDD3EA|
+------------------+
only showing top 20 rows

## User Data

Tried to bring in user data but it doesn't map to tracks in the last.fm dataset

In [7]:
user_rdd = sc.textFile("s3://msds694-final-group7/train_triplets.txt")
user_rdd = user_rdd.map(lambda x: x.split('\t')).map(lambda x: (str(x[0]), str(x[1]), int(x[2])))
schema = StructType([
    StructField('user', StringType(), True), 
    StructField('track_id', StringType(), True), 
    StructField('play_count', IntegerType(), True), 
])
user_df = ss.createDataFrame(user_rdd, schema)
user_df.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------------+------------------+----------+
|user                                    |track_id          |play_count|
+----------------------------------------+------------------+----------+
|b80344d063b5ccb3212f76538f3d9e43d87dca9e|SOAKIMP12A8C130995|1         |
|b80344d063b5ccb3212f76538f3d9e43d87dca9e|SOAPDEY12A81C210A9|1         |
|b80344d063b5ccb3212f76538f3d9e43d87dca9e|SOBBMDR12A8C13253B|2         |
|b80344d063b5ccb3212f76538f3d9e43d87dca9e|SOBFNSP12AF72A0E22|1         |
|b80344d063b5ccb3212f76538f3d9e43d87dca9e|SOBFOVM12A58A7D494|1         |
|b80344d063b5ccb3212f76538f3d9e43d87dca9e|SOBNZDC12A6D4FC103|1         |
|b80344d063b5ccb3212f76538f3d9e43d87dca9e|SOBSUJE12A6D4F8CF5|2         |
|b80344d063b5ccb3212f76538f3d9e43d87dca9e|SOBVFZR12A6D4F8AE3|1         |
|b80344d063b5ccb3212f76538f3d9e43d87dca9e|SOBXALG12A8C13C108|1         |
|b80344d063b5ccb3212f76538f3d9e43d87dca9e|SOBXHDL12A81C204C0|1         |
|b80344d063b5ccb3212f76538f3d9e43d87dca9e|SOBYHAJ12

In [96]:
s = "<HI>"
s.split("<")[1].split(">")[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'HI'

In [8]:
# Get pairs of song to track_id
song_track_id = ss.read.csv('s3://msds694-final-group7/sid_mismatches.txt', header=False)
song_track_id

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[_c0: string]

In [16]:
def divide_Str_first(x): 
    return x.split("<")[1].split(">")[0].split(" ")[0]
def divide_Str_second(x): 
    return x.split("<")[1].split(">")[0].split(" ")[1]
divide_1 = udf(divide_Str_first)
divide_2 = udf(divide_Str_second)
song_track_preprocess = song_track_id.withColumn("song_id", divide_1("_c0"))\
                                    .withColumn("track_id_1", divide_2("_c0"))\
                                    .select(["song_id", "track_id_1"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
song_track_preprocess.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+
|song_id           |track_id_1        |
+------------------+------------------+
|SOUMNSI12AB0182807|TRMMGKQ128F9325E10|
|SOCMRBE12AB018C546|TRMMREB12903CEB1B1|
|SOLPHZY12AC468ABA8|TRMMBOC12903CEB46E|
|SONGHTM12A8C1374EF|TRMMITP128F425D8D0|
|SONGXCA12A8C13E82E|TRMMAYZ128F429ECE6|
|SOMBCRC12A67ADA435|TRMMNVU128EF343EED|
|SOTDWDK12A8C13617B|TRMMNCZ128F426FF0E|
|SOEBURP12AB018C2FB|TRMMPBS12903CE90E1|
|SOSRJHS12A6D4FDAA3|TRMWMEL128F421DA68|
|SOIYAAQ12A6D4F954A|TRMWHRI128F147EA8E|
|SOUQUOG12A6D4F5965|TRMWCQW128F147DE05|
|SORRXFX12A8C141030|TRMWRQM128F92CCC1C|
|SOBWQMY12AB018AEAD|TRMWBQO128F935220C|
|SOCYTXQ12AB018E7C9|TRMWQBZ12903CC6AEF|
|SOKMSQN12A6D4FA08E|TRMWQNL128F92C6A25|
|SOFXDWK12A58A77B03|TRMWAQB128F428764C|
|SOBPEIN12AB018113A|TRMWPYJ128F931C084|
|SOANRSQ12A81C2147D|TRMWDXD128F42554E8|
|SOVWUNG12A8C137891|TRMGMLW128F426A200|
|SOOBZPE12AC468DFAF|TRMGWMD12903D03949|
+------------------+------------------+
only showing top 20 rows

In [19]:
song_to_track = user_df.join(song_track_preprocess, user_df['Track_ID'] == song_track_preprocess['song_id'])
song_to_track = song_to_track.select("user", "track_id", "play_count")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Decide not to use this dataframe because there is no track_id that maps to the track_id from last.fm

In [21]:
last_fm_play_counts = song_to_track.join(train, train['Track_ID'] == song_to_track['track_id'])
last_fm_play_counts.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+----------+--------+-----+------+-----+-----+-----+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+
|user|track_id|play_count|Track_ID|Title|Artist|Tag_1|Tag_2|Tag_3|Similar_1|Similar_2|Similar_3|Similar_4|Similar_5|Similar_6|Similar_7|Similar_8|Similar_9|Similar_10|
+----+--------+----------+--------+-----+------+-----+-----+-----+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+
+----+--------+----------+--------+-----+------+-----+-----+-----+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+